# Langchain (v0.3) 의 주요 Components
https://python.langchain.com/docs/how_to/#components

1. Chat models
- Chat model 은 '메시지'를 입력받아 출력하는 형태의 언어 모델. 

1. Messages

1. Prompt templates

1. Example selectors

1. LLMs

1. Output parsers

1. Document loaders

1. Text splitters

1. Embedding models

1. Vector stores

1. Retrievers


1. Chains
https://python.langchain.com/docs/versions/migrating_chains/


1. Agents

1. Memory

1. Callbakcs


# prompts
https://python.langchain.com/api_reference/core/prompts.html#module-langchain_core.prompts

In [ ]:
"""
prompts class 계층도

BasePromptTemplate --> PipelinePromptTemplate
                       StringPromptTemplate --> PromptTemplate
                                                FewShotPromptTemplate
                                                FewShotPromptWithTemplates
                       BaseChatPromptTemplate --> AutoGPTPrompt
                                                  ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



BaseMessagePromptTemplate --> MessagesPlaceholder
                              BaseStringMessagePromptTemplate --> ChatMessagePromptTemplate
                                                                  HumanMessagePromptTemplate
                                                                  AIMessagePromptTemplate
                                                                  SystemMessagePromptTemplate
"""
None

# API Key 가져오기

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
print(f'{os.environ['OPENAI_API_KEY'][:20]}...')

sk-proj-iKU13YeoxNgF...


# 1. FewShotPromptTemplate

In [4]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [5]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## 0.PromptTemplate

In [6]:
# 방식1
prompt = PromptTemplate.from_template("What is the capital of {country}")
prompt.format(country='France')

'What is the capital of France'

In [ ]:
# 템플릿을 만드는 이점중 하나는 '유효성 검사' 를 할 수 있다는 점이다
# 가령 아래와 같이 하면 에러다

# t.format()  # country= 를 빼먹으면 KeyError: 'country'

In [ ]:
# 템플릿을 만드는 이점 또 한가지!
# prompt template 을 디스크에 '저장'하고 'load' 할 수 있기 떼문이다.

# 나중에 LLM 다룰때 prompt 는 매우 중요합니다.
# 대규모 프로젝트에서는 
# - 'prompt 만 만드는 팀'이 있고
# - '코딩하는 팀'이 따로 있을것이다.  
#    데이터베이스나 파일 등에 만들어 저장해 놓은 prompt 를 load 해야 할겁니다

In [7]:
# 방식2
prompt2 = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=['country'],
)

prompt2.format(country='Korea')

'What is the capital of Korea'

## 1.FewShotPromptTemplate
모델에 예제(sample) 주기

In [8]:
# 모델에게 '어떻게 대답해야 하는 지에 대한 예제(example)'를 AI 모델에게 주는 것이
# prompt 를 사용해서 '어떻게 대답해야 하는지 알려주는 것'보다 훨씬 좋다

# FewShotPromptTemplate 이 하는 일이 바로 그거다!
# - 이를 통해 예제(샘플)를 형식화(포맷) 할수 있다.
# - 이런 예제들을 데이터베이스등에 저장시켜놓고 활용할수도 있다

In [9]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate

In [10]:
# 모델이 나에게 '이런 식으로 답변해 줬으면 좋겠다' 라고 제시하는 example(예제) 들
examples = [
    {
        "question": "What do you know about France?",
        # ↓ 원하는 형식의 답변.
        "answer": """
              Here is what I know:
              Capital: Paris
              Language: French
              Food: Wine and Cheese
              Currency: Euro
              """,
    },
   {
    "question": "What do you know about Italy?",
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
    "question": "What do you know about Greece?",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },   
]

In [11]:
# 일단, 예제(example) 없이 전달하면?
chat.invoke("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of over 67 million people. The official language is French, and the currency is the Euro.

France is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, countryside, and coastline. It is also known for its wine and cheese production, fashion industry, and art and literature.

France has a long history of political and cultural influence, including the French Revolution, the Napoleonic Wars, and the establishment of the French colonial empire. Today, France is a member of the United Nations, NATO, and the European Union, among other international organizations.

AIMessage(content='France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of over 67 million people. The official language is French, and the currency is the Euro.\n\nFrance is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, countryside, and coastline. It is also known for its wine and cheese production, fashion industry, and art and literature.\n\nFrance has a long history of political and cultural influence, including the French Revolution, the Napoleonic Wars, and the establishment of the French colonial empire. Today, France is a member of the United Nations, NATO, and the European Union, among other international organizations.', additi

In [12]:
example_template = """
    Human: {question},
    AI: {answer},
"""
# ↑ {question} 과 {answer} 는 위 샘플과 동일한 key를 사용하여 작성한다.
None

In [13]:
example_prompt = PromptTemplate.from_template(example_template)
example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question},\n    AI: {answer},\n')

In [14]:
print(example_prompt.format(**examples[2]))


    Human: What do you know about Greece?,
    AI: 
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      ,



In [16]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,  # 사용할 prompt
    examples = examples,  # 준비된 예제들

    # suffix=
    # 포맷팅된 모든 예제 마지막에 나오는 내용.
    suffix="Human: What do you know abount {country}?",

    # 어떤 변수를 suffix 에서 사용할지 지정
    input_variables=['country']
)

prompt

FewShotPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, examples=[{'question': 'What do you know about France?', 'answer': '\n              Here is what I know:\n              Capital: Paris\n              Language: French\n              Food: Wine and Cheese\n              Currency: Euro\n              '}, {'question': 'What do you know about Italy?', 'answer': '\n      I know this:\n      Capital: Rome\n      Language: Italian\n      Food: Pizza and Pasta\n      Currency: Euro\n      '}, {'question': 'What do you know about Greece?', 'answer': '\n      I know this:\n      Capital: Athens\n      Language: Greek\n      Food: Souvlaki and Feta Cheese\n      Currency: Euro\n      '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question},\n    AI: {answer},\n'), suffix='Human: What do you know abount {country}?')

In [17]:
print(prompt.format(country='Germany'))


    Human: What do you know about France?,
    AI: 
              Here is what I know:
              Capital: Paris
              Language: French
              Food: Wine and Cheese
              Currency: Euro
              ,



    Human: What do you know about Italy?,
    AI: 
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      ,



    Human: What do you know about Greece?,
    AI: 
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      ,


Human: What do you know abount Germany?


In [ ]:
# step1  example 리스트를 만들고   examples
# step2  FewShotPromptTemplate 에 전달했고 examples=
# step3  어떻게 전달한 예제들을 형식화 할지 알려주었고
# step4  마지막에 질문을 포함시켰다.  suffix, input_variables

# AI 는 우리의 예제들과 똑같은 구조, 형태로 답변하게 될겁니다

In [18]:
chain = prompt | chat

chain

FewShotPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, examples=[{'question': 'What do you know about France?', 'answer': '\n              Here is what I know:\n              Capital: Paris\n              Language: French\n              Food: Wine and Cheese\n              Currency: Euro\n              '}, {'question': 'What do you know about Italy?', 'answer': '\n      I know this:\n      Capital: Rome\n      Language: Italian\n      Food: Pizza and Pasta\n      Currency: Euro\n      '}, {'question': 'What do you know about Greece?', 'answer': '\n      I know this:\n      Capital: Athens\n      Language: Greek\n      Food: Souvlaki and Feta Cheese\n      Currency: Euro\n      '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question},\n    AI: {answer},\n'), suffix='Human: What do you know abount {country}?')
| ChatOpenAI(callbacks=[<langchain_core.callbacks.stream

In [20]:
chain.invoke({
    'country': "Germany"
})

AI: 
      I know this:
      Capital: Berlin
      Language: German
      Food: Bratwurst and Sauerkraut
      Currency: Euro

AIMessage(content='AI: \n      I know this:\n      Capital: Berlin\n      Language: German\n      Food: Bratwurst and Sauerkraut\n      Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--4ac301e6-f283-49d7-8efc-3d66310c661f-0', usage_metadata={'input_tokens': 156, 'output_tokens': 35, 'total_tokens': 191, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [21]:
chain.invoke({
    'country': "Turkey"
})

AI: 
      I know this:
      Capital: Ankara
      Language: Turkish
      Food: Kebab and Baklava
      Currency: Turkish Lira

AIMessage(content='AI: \n      I know this:\n      Capital: Ankara\n      Language: Turkish\n      Food: Kebab and Baklava\n      Currency: Turkish Lira', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--3bc65125-927d-4958-b113-50244aad889d-0', usage_metadata={'input_tokens': 156, 'output_tokens': 35, 'total_tokens': 191, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 2.FewShowChatMessagePromptTemplate

In [22]:
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate

from langchain_core.prompts.chat import ChatPromptTemplate

In [23]:
# 모델이 나에게 '이런 식으로 답변해 줬으면 좋겠다' 라고 제시하는 example(예제) 들
examples = [
    {
        "country": "France",
        "answer": """
              Here is what I know:
              Capital: Paris
              Language: French
              Food: Wine and Cheese
              Currency: Euro
              """,
    },
   {
    "country": "Italy",
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
    "country": "Greece",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },   
]

In [24]:
prompt = ChatPromptTemplate.from_messages([
    ('human', "What do you know about {country}?"),  # 반드시 example 과 동일한 key 로!
    ('ai', "{answer}"),  # ai 의 답변들을 여기에 넣어줄거다.
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = prompt,
    examples = examples
    # suffix= 필요 없다.
)

final_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a geography expert, you give short answers."),
    example_prompt,
    ('human', 'What do you know about {country}?'),  # 질문
])


In [25]:
final_prompt.format_messages(country='Germany')

[SystemMessage(content='You are a geography expert, you give short answers.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What do you know about France?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n              Here is what I know:\n              Capital: Paris\n              Language: French\n              Food: Wine and Cheese\n              Currency: Euro\n              ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What do you know about Italy?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n      I know this:\n      Capital: Rome\n      Language: Italian\n      Food: Pizza and Pasta\n      Currency: Euro\n      ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What do you know about Greece?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n      I know this:\n      Capital: Athens\n      Language: Greek\n      Food: Souvlaki and Feta Cheese\n      C

In [26]:
chain = final_prompt | chat

In [27]:
chain.invoke({'country': "Finland"})


      I know this:
      Capital: Helsinki
      Language: Finnish, Swedish
      Food: Rye bread and salmon
      Currency: Euro
      

AIMessage(content='\n      I know this:\n      Capital: Helsinki\n      Language: Finnish, Swedish\n      Food: Rye bread and salmon\n      Currency: Euro\n      ', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--e5ba4ce9-40e6-49eb-a9e5-0d9afe835c59-0', usage_metadata={'input_tokens': 167, 'output_tokens': 33, 'total_tokens': 200, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [28]:
chain.invoke({'country': "South Korea"})


      I know this:
      Capital: Seoul
      Language: Korean
      Food: Kimchi and Bibimbap
      Currency: South Korean Won
      

AIMessage(content='\n      I know this:\n      Capital: Seoul\n      Language: Korean\n      Food: Kimchi and Bibimbap\n      Currency: South Korean Won\n      ', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--6e9cec5f-b900-4bfb-938d-43af102adb66-0', usage_metadata={'input_tokens': 168, 'output_tokens': 34, 'total_tokens': 202, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})